In [1]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import time
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=100,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

In [ ]:
test_df = pd.read_csv("data/test_dataset.csv")

In [5]:
def predict_intent(text):
    prompt = f"""
Aşağıdaki cümleyi incele ve en uygun intent etiketini sadece tek kelime olarak ver:
-selamlama
-veda
-ana_ogun
-ara_ogun
-tatli
-kahvalti
-malzeme_sorgu
-diyet_onerisi
-konu_disi

Cümle: "{text}"
Intent:
"""
    try:
        response = llm.invoke(prompt)
        return response.content.strip().lower()
    except Exception as e:
        print("Hata:", e)
        return "bilinmiyor"

In [6]:
# Gerçek ve tahmin edilen etiketleri karşılaştır
y_true = test_df["intent"].tolist()
y_pred = []

for i, row in test_df.iterrows():
    intent = predict_intent(row["text"])
    y_pred.append(intent)
    time.sleep(3)  # OpenAI API rate limit için bekleme

# Performans metriklerini yazdır
print("OpenAI Classification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

print("OpenAI Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

OpenAI Classification Report:
               precision    recall  f1-score   support

     ana_ogun       1.00      0.97      0.98        30
     ara_ogun       0.94      1.00      0.97        30
diyet_onerisi       0.96      0.77      0.85        30
     kahvalti       0.83      1.00      0.91        30
    konu_disi       1.00      0.90      0.95        30
malzeme_sorgu       1.00      1.00      1.00        30
    selamlama       0.81      1.00      0.90        30
        tatli       1.00      0.97      0.98        30
         veda       1.00      0.87      0.93        30

     accuracy                           0.94       270
    macro avg       0.95      0.94      0.94       270
 weighted avg       0.95      0.94      0.94       270

OpenAI Confusion Matrix:
[[29  1  0  0  0  0  0  0  0]
 [ 0 30  0  0  0  0  0  0  0]
 [ 0  1 23  6  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0]
 [ 0  0  0  0 27  0  3  0  0]
 [ 0  0  0  0  0 30  0  0  0]
 [ 0  0  0  0  0  0 30  0  0]
 [ 0  0  1  0  0 